Author: Roel Faber

Goal of this script: take the raw matchdata from Eredivisiedata.ipynb and use it to create useful features, such as match history, standings, form, etc.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set()

In [40]:
df = pd.read_csv('Data/matches.csv',index_col=0)
df.head()
df_20192020 = pd.read_csv('Data/matches_20192020.csv',index_col=0)

# Add last season of football to data

In [41]:
df = df[df['Season']!='2019-2020'].copy()

df = df.append(df_20192020).copy()

# Compute columns

In [42]:
df['GoalsScored'] = df['HomeGoals'] + df['AwayGoals']

In [43]:
df['HomeAdvantage'] = df['HomeGoals'] - df['AwayGoals']

In [44]:
df

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,HomeAdvantage
0,1959-1960,1,ADO Den Haag,Willem II,3,3,3,1,1,6,0
1,1959-1960,1,AFC Ajax,NAC Breda,3,0,1,3,0,3,3
2,1959-1960,1,Blauw Wit,Sittardia,5,2,1,3,0,7,3
3,1959-1960,1,USV Elinkwijk,MVV Maastricht,2,0,1,3,0,2,2
4,1959-1960,1,Feijenoord,Sparta Rotterdam,0,1,2,0,3,1,-1
...,...,...,...,...,...,...,...,...,...,...,...
227,2019-2020,26,sc Heerenveen,AFC Ajax,1,3,2,0,3,4,-2
228,2019-2020,26,FC Utrecht,Sparta Rotterdam,5,1,1,3,0,6,4
229,2019-2020,26,Feyenoord,Willem II,2,0,1,3,0,2,2
230,2019-2020,26,Heracles Almelo,RKC Waalwijk,4,2,1,3,0,6,2


In [45]:
df

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,HomeAdvantage
0,1959-1960,1,ADO Den Haag,Willem II,3,3,3,1,1,6,0
1,1959-1960,1,AFC Ajax,NAC Breda,3,0,1,3,0,3,3
2,1959-1960,1,Blauw Wit,Sittardia,5,2,1,3,0,7,3
3,1959-1960,1,USV Elinkwijk,MVV Maastricht,2,0,1,3,0,2,2
4,1959-1960,1,Feijenoord,Sparta Rotterdam,0,1,2,0,3,1,-1
...,...,...,...,...,...,...,...,...,...,...,...
227,2019-2020,26,sc Heerenveen,AFC Ajax,1,3,2,0,3,4,-2
228,2019-2020,26,FC Utrecht,Sparta Rotterdam,5,1,1,3,0,6,4
229,2019-2020,26,Feyenoord,Willem II,2,0,1,3,0,2,2
230,2019-2020,26,Heracles Almelo,RKC Waalwijk,4,2,1,3,0,6,2


# Combine teams that have changed names

Evt. nog uitwerken: dan functie schrijven om te checken of bepaalde teams samen voorkomen in seizoenen.

In [46]:
teamlist = df['Home'].unique()
teamlist

array(['ADO Den Haag', 'AFC Ajax', 'Blauw Wit', 'USV Elinkwijk',
       'Feijenoord', "Fortuna '54", 'Rapid JC Heerlen', 'FC Volendam',
       'VVV-Venlo', 'DOS Utrecht', 'DWS Amsterdam', 'MVV Maastricht',
       'NAC Breda', 'PSV', 'Sportclub Enschede', 'Sittardia',
       'Sparta Rotterdam', 'Willem II', "VV Alkmaar '54", 'Groninger VAV',
       'NOAD Tilburg', 'De Volewijckers', 'Heracles Almelo',
       'Go Ahead Eagles', 'Telstar', 'FC Twente', 'GVAV Groningen',
       'FC Xerxes', 'NEC Nijmegen', 'Xerxes/D.H.C.', "AZ '67 Alkmaar",
       'FSC Geleen', 'Holland Sport', 'SVV Schiedam', 'HFC Haarlem',
       'SBV Excelsior', 'FC Utrecht', 'FC Den Haag-ADO', 'FC Groningen',
       "FC Den Bosch '67", 'Vitesse', 'FC Amsterdam', 'Feyenoord',
       'Roda JC Kerkrade', 'De Graafschap', 'WVV Wageningen',
       'SC Eindhoven', 'FC Den Haag', 'PEC Zwolle', 'FC Wageningen',
       'SBV Haarlem', "PEC Zwolle '82", 'Fortuna Sittard',
       'Helmond Sport', "Drechtsteden '79", 'AZ Alkmaar', 

In [63]:
# Manually map teams with changed names to most recent name

rename_teams = {'Feijenoord':'Feyenoord',
               "Fortuna '54":'Fortuna Sittard', # Ook fusie met Sittardia, maar zaten gelijktijdig in Eredivisie
               'FSC Geleen':'Fortuna Sittard', 
               'DOS':'FC Utrecht',
                'USV Elinkwijk':'FC Utrecht',
                'Velox':'FC Utrecht',
               'Sportclub Enschede':'FC Twente',
               "VV Alkmaar '54":'AZ Alkmaar',
               "AZ '67 Alkmaar":'AZ Alkmaar',
               'GVAV Groningen':'FC Groningen',
               'FC Den Haag-ADO':'ADO Den Haag',
               'FC Den Haag':'ADO Den Haag',
               "Drechtsteden '79":'FC Dordrecht',
               "Dordrecht '90":'FC Dordrecht',
               "S.V.V./Dordrecht '90":'FC Dordrecht',
               "FC Den Bosch '67":'FC Den Bosch',
               'BVV Den Bosch':'FC Den Bosch',
               } 

In [64]:
df = df.replace(rename_teams)

In [65]:
(df['Home'] == df['Away']).sum()

0

# Get more informative features

Features to include:

1) Standings in previous season(s)

2) Recent results against the opponent

3) Recent form (results of previous matches)

## Standings in seasons

In [66]:
standings_df = pd.DataFrame(columns=df['Season'].unique(),index=df['Home'].unique())
points_df = pd.DataFrame(columns=df['Season'].unique(),index=df['Home'].unique())

def get_standings(seasondf,seasonname):
    """Get the final results table for a season
    
    Parameters
    -------------
    seasondf, pd.DataFrame
        Dataframe containing the matches in a season
        
    seasonname, str
        string of season, should match value in seasondf
        
    Output
    -------------
    
    standings_df, pd.DataFrame
        Dataframe with standings based on points at the end of a season
    points_df, pd.DataFrame
        Dataframe with points acquired in a given season"""
    teamlist = seasondf['Home'].unique()
    standings = pd.DataFrame(index=teamlist,columns=range(1,35))
    for index, row in seasondf.iterrows():
        standings.loc[row['Home'],row['Round']] = row['HomePoints']
        standings.loc[row['Away'],row['Round']] = row['AwayPoints']
    standings['Total'] = standings.sum(axis=1)
    standings['Total'].rank(ascending=False).sort_values()
    standings['Final'] = standings['Total'].rank(ascending=False).sort_values()
    for index,row in standings.iterrows():
        standings_df.at[index,seasonname] = row['Final']
        points_df.at[index,seasonname] = row['Total']

In [67]:
%%time
for season in df['Season'].unique():
    seasondf = df[df['Season']==season]
    get_standings(seasondf,season)

Wall time: 5.61 s


In [68]:
standings_df.to_csv('Data/final_standings.csv')
points_df.to_csv('Data/final_points.csv')

In [69]:
standings_df = pd.read_csv('Data/final_standings.csv')
points_df = pd.read_csv('Data/final_points.csv')

## Add information to matches df

In [70]:
df['Home_standings_last_season'] = np.nan
df['Home_standings_this_season'] = np.nan
df['Home_standings_form'] = np.nan
df['Home_points_last_season'] = np.nan
df['Home_points_this_season'] = np.nan
df['Away_standings_last_season'] = np.nan
df['Away_standings_this_season'] = np.nan
df['Away_standings_form'] = np.nan
df['Away_points_last_season'] = np.nan
df['Away_points_this_season'] = np.nan

In [71]:
df.head()

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,...,Home_standings_last_season,Home_standings_this_season,Home_standings_form,Home_points_last_season,Home_points_this_season,Away_standings_last_season,Away_standings_this_season,Away_standings_form,Away_points_last_season,Away_points_this_season
0,1959-1960,1,ADO Den Haag,Willem II,3,3,3,1,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1959-1960,1,AFC Ajax,NAC Breda,3,0,1,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1959-1960,1,Blauw Wit,Sittardia,5,2,1,3,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1959-1960,1,FC Utrecht,MVV Maastricht,2,0,1,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1959-1960,1,Feyenoord,Sparta Rotterdam,0,1,2,0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add standings

In [72]:
standings_df.head()

,Unnamed: 0,1959-1960,1960-1961,1961-1962,1962-1963,1963-1964,1964-1965,1965-1966,1966-1967,1967-1968,...,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020
0,ADO Den Haag,8.0,9.0,14.5,12.0,7.5,3.5,3.0,4.0,4.0,...,7.0,15.0,9.0,9.0,13.5,11.5,11.0,7.0,8.5,17.0
1,AFC Ajax,1.0,2.0,4.0,2.5,6.0,12.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.5
2,Blauw Wit,7.0,NaN,3.0,10.0,15.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FC Utrecht,15.0,15.0,NaN,NaN,NaN,NaN,14.0,17.0,NaN,...,9.0,11.0,5.0,10.0,11.5,5.0,4.0,5.0,5.5,6.5
4,Feyenoord,3.0,1.0,1.5,4.0,3.5,1.0,2.0,2.0,2.0,...,10.0,2.0,2.5,2.0,4.0,3.0,1.0,4.0,3.0,3.0


## Results from last 5 matches

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18247 entries, 0 to 231
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 18247 non-null  object 
 1   Round                  18247 non-null  int64  
 2   Home                   18247 non-null  object 
 3   Away                   18247 non-null  object 
 4   HomeGoals              18247 non-null  int64  
 5   AwayGoals              18247 non-null  int64  
 6   Result                 18247 non-null  int64  
 7   HomePoints             18247 non-null  int64  
 8   AwayPoints             18247 non-null  int64  
 9   GoalsScored            18247 non-null  int64  
 10  HomeAdvantage          18247 non-null  int64  
 11  standings_last_season  0 non-null      float64
 12  standings_this_season  0 non-null      float64
 13  standings_form         0 non-null      float64
 14  points_last_season     0 non-null      float64
 15  poin

In [28]:
def get_form(df, team, window, outputcol):
    teamdf = df.loc[(df.Home==team) | (df.Away==team)]
    pointslist = []
    for index, row in df.iterrows():
        if row.Home==team:
            pointslist.append(row.HomePoints)
        elif row.Away==team:
            pointslist.append(row.AwayPoints)
    teamdf.loc[:,'TeamPoints'] = pointslist
    teamdf.loc[:,outputcol] = teamdf.TeamPoints.rolling(window=window).sum()
    df.loc[teamdf[teamdf.Home==team].index,'Home_'+outputcol] = teamdf.loc[:,outputcol]
    df.loc[teamdf[teamdf.Away==team].index,'Away_'+outputcol] = teamdf.loc[:,outputcol]

In [29]:
get_form(df, 'AFC Ajax', 5, 'RecentForm')

C:\Users\roel_\anaconda3\envs\football-predictions\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\roel_\anaconda3\envs\football-predictions\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


ValueError: cannot reindex from a duplicate axis

In [ ]:
# get_form_opponent(df, team, opponent, window, outputcol):
#     Optie 1
#     ---------------
#     establish recentdf
#     pointslist
#     toevoegen aan df
#     berekenen rolling som
#     toewijzen aan alle combinaties
    
#     Optie 2
#     ----------------
#     Itereer met functie over rijen heen
    
    
    

In [ ]:
team = 'AFC Ajax'
opponent = 'PSV'

In [ ]:
teamdf = df.loc[((df.Home==team)&]

In [ ]:
teamlist = df.Home.unique()

In [ ]:
%%time
for team in teamlist:
    get_form(df, team, 5, 'RecentForm')

In [ ]:
df = df.drop('RecentForm',axis=1)

In [ ]:
df.to_csv('Data/matcheswithform.csv')

In [ ]:
df = pd.read_csv('Data/matcheswithform.csv', index_col = 0)

In [ ]:
df